In [ ]:
try:
    spark.stop()
except:
    pass
# Using findspark to find automatically the spark folder
import findspark
findspark.init()

import numpy as np

minlon,maxlon,minlat,maxlat=-179.9,180.0,-90.0,90.0

In [ ]:
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import explode
from pyspark.sql.functions import split, col
from pyspark.sql.functions import last,max

In [ ]:
spark = SparkSession.builder.appName("engine").getOrCreate()

In [ ]:
line = spark.readStream.format("socket").option("host", "localhost").option("port", 9998).load()
phone = spark.readStream.format("socket").option("host","192.168.1.142").option("port",9000).load()

In [ ]:
ppoint = phone.select(explode(split(phone.value," ")).alias("id"))
ppoint = ppoint.withColumn('lat', split(ppoint["id"], '\,')[3].cast(types.FloatType()))
ppoint = ppoint.withColumn('lon', split(ppoint["id"], '\,')[2].cast(types.FloatType()))
ppoint = ppoint.withColumn('timestamp', split(ppoint["id"], '\,')[1].cast(types.LongType()))
ppoint = ppoint.withColumn('id', split(ppoint["id"], '\,')[0])
ppoint = ppoint.groupby('id').agg(max('timestamp'), last('lat'), last('lon')).toDF('id','time','lat','lon').drop('id','time')

def batchProcess2(sdf, epoch_id):
    df = sdf.toPandas()
    if df.empty:
        pass
    else:
        global minlon
        global maxlon
        global minlat
        global maxlat
        minlon = float(df.iloc[0]['lon'] - 0.5)
        maxlon = float(df.iloc[0]['lon'] + 0.5)
        minlat = float(df.iloc[0]['lat'] - 0.5)
        maxlat = float(df.iloc[0]['lat'] + 0.5)

In [ ]:
point = line.select(explode(split(line.value," ")).alias("lon"))
point = point.withColumn('lat', split(point["lon"], '\,')[1].cast(types.FloatType()))
point = point.withColumn('lon', split(point["lon"], '\,')[0].cast(types.FloatType()))
point = point.groupby('lat','lon').count()
def batchProcess1(sdf, epoch_id):
    global minlon
    global maxlon
    global minlat
    global maxlat
    df = sdf.toPandas()
    df1 = df[(df['lat']>=minlat)&(df['lat']<=maxlat)&(df['lon']>=minlon)&(df['lon']<=maxlon)]
    f = open('tmp','w')
    f.write(str(np.sum(df1['count'].to_numpy()))+"\n")
    f.close()

In [ ]:
ppoint.writeStream.outputMode("complete").foreachBatch(batchProcess2).start()
point.writeStream.outputMode("complete").foreachBatch(batchProcess1).start()